# Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_circles, make_moons, make_swiss_roll
from sklearn import preprocessing
from functools import reduce #To use multiple np.kron

from scipy.optimize import curve_fit

import math


# Datasets

## Spiral function

In [ ]:

# Function to generate a spiral
def make_spiral(phase, slope, noise_std, num_points, turns, label, random_seed=0):
    theta = np.linspace(0, 2 * np.pi * turns, num_points)
    r = slope *(theta ) # Spiral grows in radius with angle

    # Add random noise in theta
    np.random.seed(random_seed)
    theta += np.random.normal(0, noise_std, num_points)

    x = r * np.cos(theta + phase)
    y = r * np.sin(theta + phase)

    # Add Gaussian noise
    #x += np.random.normal(0, noise_std, num_points)
    #y += np.random.normal(0, noise_std, num_points)

    labels = np.full(num_points, label)
    return x, y, labels


def generate_spirals(num_points, random_seed, noise = 0.5, plot = False, test_size=0.25):
    # Parameters
    turns = 1  # Number of turns in the spiral
    slope = 1

    # Generate two spirals
    x1, y1, labels1 = make_spiral(0, slope, noise, num_points, turns, label=0, random_seed=random_seed)
    x2, y2, labels2 = make_spiral(np.pi, slope, noise, num_points, turns, label=1, random_seed=random_seed+1)

    # Create a DataFrame
    df = pd.DataFrame({
        'x': np.concatenate([x1, x2]),
        'y': np.concatenate([y1, y2]),
        'label': np.concatenate([labels1, labels2])
    })

    # Scale it to a range
    min_max_scaler = preprocessing.MinMaxScaler()
    df[['x','y']] =  min_max_scaler.fit_transform(df[['x','y']])

    # Split the data into training and test sets
    train_df, test_df = train_test_split(df, test_size=test_size, random_state=42*random_seed)

    train_arr = np.array(train_df[['x','y']])
    train_lb = np.array(train_df['label'])
    test_arr = np.array(test_df[['x','y']])
    test_lb = np.array(test_df['label'])

    if plot:
        # Plot the training and test data for visualization
        plt.scatter(train_df['x'], train_df['y'], c=train_df['label'], label='Train Data', alpha=0.6)
        plt.scatter(test_df['x'], test_df['y'], c=test_df['label'], marker='x', label='Test Data', alpha=0.6)
        plt.legend()
        plt.title('Training and Test Data for Two Spirals')

    return train_arr, train_lb, test_arr, test_lb


#num_points = 200
#random_seed = 1
#train_arr, train_lb, test_arr, test_lb, df = generate_spirals(num_points,random_seed, plot= True)

## Moons

In [ ]:
def generate_moons(num_points, random_seed,  noise = 0.1, plot = False, test_size=0.25):
    # Generate datasets
    X_moons, y_moons = make_moons(n_samples=num_points, noise=noise, random_state=random_seed)

    # Scale it to a range
    min_max_scaler = preprocessing.MinMaxScaler()
    X_moons = min_max_scaler.fit_transform(X_moons)


    # Split the data into training and test sets
    X_train_moons, X_test_moons, y_train_moons, y_test_moons = train_test_split(
        X_moons, y_moons, test_size=test_size, random_state=42*random_seed
    )

    if plot:
        # Plot the training and test data for visualization
        plt.scatter(
            X_train_moons[:, 0],
            X_train_moons[:, 1],
            c=y_train_moons,
            label="Train Data",
            alpha=0.6,
        )
        plt.scatter(
            X_test_moons[:, 0],
            X_test_moons[:, 1],
            c=y_test_moons,
            marker="x",
            label="Test Data",
            alpha=0.6,
        )
        plt.legend()
        plt.title("Training and Test Data for Moons Dataset")
        plt.show()
    return X_train_moons, y_train_moons, X_test_moons, y_test_moons

#num_points = 200
#random_seed = 1
#X_train_moons, y_train_moons, X_test_moons, y_test_moons = generate_moons(num_points,random_seed, plot= True)

## Circles

In [ ]:
def generate_circles(num_points, random_seed,  noise = 0.1, plot = False, test_size=0.25, factor=0.5):
    # Generate datasets
    X_circles, y_circles = make_circles(n_samples=num_points, noise=noise, random_state=random_seed, factor=factor)

    # Scale it between 0 and 1
    min_max_scaler = preprocessing.MinMaxScaler()
    X_circles = min_max_scaler.fit_transform(X_circles)

    # Split the data into training and test sets
    X_train_circles, X_test_circles, y_train_circles, y_test_circles = train_test_split(
        X_circles, y_circles, test_size=test_size, random_state=42*random_seed)

    if plot:
        # Plot the training and test data for visualization
        plt.scatter(
            X_train_circles[:, 0],
            X_train_circles[:, 1],
            c=y_train_circles,
            label="Train Data",
            alpha=0.6,
        )
        plt.scatter(
            X_test_circles[:, 0],
            X_test_circles[:, 1],
            c=y_test_circles,
            marker="x",
            label="Test Data",
            alpha=0.6,
        )
        plt.legend()
        plt.title("Training and Test Data for Circles Dataset")
        plt.show()
    return X_train_circles, y_train_circles, X_test_circles, y_test_circles

# Diminish size

In [ ]:
def diminish_size(xy_array, scale = 0.5):
    if xy_array.shape[0] <=1:
      print('No diminish')
      return
    if xy_array.shape[1] != 2:
      print('Wrong dimension')
      return
    max_x = np.max(xy_array[:,0])
    min_x = np.min(xy_array[:,0])
    max_y = np.max(xy_array[:,1])
    min_y = np.min(xy_array[:,1])
    # Rescaling
    xy_array[:,0] = (xy_array[:,0] - min_x)/(max_x - min_x) * scale
    xy_array[:,1] = (xy_array[:,1] - min_y)/(max_y - min_y) * scale
    # Recentering
    xy_array[:,0] = xy_array[:,0] + (1-scale)/2
    xy_array[:,1] = xy_array[:,1] + (1-scale)/2
    return xy_array

# Quantum Feature Map

In [ ]:
def softmax_gaussian(x,dim_qm,beta): # It assumes data is between 0 and 1
    alpha =  np.arange(dim_qm) / (dim_qm-1)
    P = np.array([np.sqrt(np.exp(-beta*np.square(alpha-x_i)) / np.sum(np.exp(-beta*np.square(alpha-x_i)))) for x_i in x])
    #dimshape = [P.shape[1]]*P.shape[0]
    #print(P.shape)
    ##reduce(np.kron, P).reshape(dimshape) # reshape Generates error with np.outer among others
    Psi = reduce(np.kron, P)
    Psi = Psi / np.linalg.norm(Psi)
    return Psi

#softmax_gaussian(train_arr[0],100,5)

#print(softmax_gaussian(np.array([0.1,0.2]),7,5).shape)
#print(softmax_gaussian(np.array([1,2]),7,5))
np.linalg.norm(softmax_gaussian(np.array([1,5.2]),7,5))

#beta = 5
#dim_qm = 100
#x = np.arange(dim_qm)
#alpha =  np.arange(dim_qm) / (dim_qm-1)
#P = np.array([np.sqrt(np.exp(-beta*np.square(alpha-x_i))/sum(np.exp(-beta*np.square(alpha-x_i)))) for x_i in x])
#reduce(np.kron, P)

1.0

In [ ]:
#math.factorial(0)

In [ ]:
#np.arange(6).reshape(2,3).T

In [ ]:
def squeezed(phi_input,trunc_n, r):
    phi = phi_input #*np.pi
    Psi = 1/np.sqrt(np.cosh(r)) * np.array([np.sqrt(float(math.factorial(2*n)))
      /(float(2**n) * float(math.factorial(n)))*np.exp(n*(phi)*1j) * np.power(np.tanh(r),n)*(-1)**n for n in range(trunc_n)])
    #print(Psi.shape)
    Psi = Psi.T # To make the tensorial product along the right dimension
    #dimshape = [Psi.shape[1]]*Psi.shape[0]
    tensorial_prod = reduce(np.kron, Psi)
    return tensorial_prod / np.linalg.norm(tensorial_prod) ##.reshape(dimshape) # reshape Generates error with np.outer among others

#squeezed(1,np.array([0,np.pi,np.pi/3]),7)

In [ ]:
#print(np.random.rand(*np.array([[1,2],[3,4]]).shape))

In [ ]:
"""
def coherent(x, trunc_n, gamma):
    # Assuming data is between 0 and 1
    theta = x*np.pi
    #theta = np.random.rand(*x.shape)*np.pi #np.arcsin(x)
    alpha = x* np.exp(1j*theta)
    Psi = np.array([np.exp(-gamma * abs(alpha)**2 /2) * alpha**n * np.float_power(gamma,n/2)/
                    np.sqrt(float(math.factorial(n))) for n in range(trunc_n)])
    Psi = Psi.T # To make the tensorial product along the right dimension
    tensor_prod = reduce(np.kron, Psi)
    #dimshape = [Psi.shape[1]]*Psi.shape[0]
    return tensor_prod/np.linalg.norm(tensor_prod)



state = coherent(np.array([0,0.2]),2,5)
print(state.shape)
print(state)
print(np.linalg.norm(state))"""

'\ndef coherent(x, trunc_n, gamma):\n    # Assuming data is between 0 and 1\n    theta = x*np.pi\n    #theta = np.random.rand(*x.shape)*np.pi #np.arcsin(x)\n    alpha = x* np.exp(1j*theta)\n    Psi = np.array([np.exp(-gamma * abs(alpha)**2 /2) * alpha**n * np.float_power(gamma,n/2)/\n                    np.sqrt(float(math.factorial(n))) for n in range(trunc_n)])\n    Psi = Psi.T # To make the tensorial product along the right dimension\n    tensor_prod = reduce(np.kron, Psi)\n    #dimshape = [Psi.shape[1]]*Psi.shape[0]\n    return tensor_prod/np.linalg.norm(tensor_prod)\n\n\n\nstate = coherent(np.array([0,0.2]),2,5)\nprint(state.shape)\nprint(state)\nprint(np.linalg.norm(state))'

In [ ]:
def coherent(x,dim,beta):
    ''' Represents each state with a coherent state
    '''
    alpha = lambda x: x * (np.cos(np.pi * x) + 1j * np.sin(np.pi * x))
    basis = np.arange(dim)
    P = np.array([np.power(alpha(x) * np.sqrt(beta),n)/
                  np.sqrt(float(math.factorial(n))) for n in basis]).T
    Psi = reduce(np.kron, P)
    Psi = Psi / np.linalg.norm(Psi)
    return Psi

In [ ]:
def onehot(y,y_set = [0,1]):
    '''Quantum Feature Map for discrete variables, i.e. labels
    y: label
    y_set: set of labels
    Each variable in y_set is an element of the basis'''

    return np.array([int(i == y) for i in y_set])

#onehot(0)

# States and Matrices

In [ ]:
def pure_density(train_QFM):
    state = np.sum(train_QFM,axis=0)
    norm_state = state/np.linalg.norm(state)
    density = np.outer(norm_state,np.conjugate(norm_state))
    return density

def pure_state(train_QFM):
    state = np.sum(train_QFM,axis=0)
    norm_state = state/np.linalg.norm(state)
    return norm_state


In [ ]:
"""def mixed_density(train_QFM):
    Psi_sum=np.zeros((np.shape(train_QFM)[1], np.shape(train_QFM)[1]), dtype=np.complex128)
    for i in range(len(train_QFM)):
        norm_state = train_QFM[i]/np.linalg.norm(train_QFM[i])
        Psi_sum += np.outer(norm_state,np.conj(norm_state))
    #state = np.sum(train_QFM,axis=0)
    #norm_state = state/np.sum(state)
    density = Psi_sum/len(train_QFM)
    return density

print(np.shape(np.outer(np.zeros(3),np.zeros(3))))
"""

'def mixed_density(train_QFM):\n    Psi_sum=np.zeros((np.shape(train_QFM)[1], np.shape(train_QFM)[1]), dtype=np.complex128)\n    for i in range(len(train_QFM)):\n        norm_state = train_QFM[i]/np.linalg.norm(train_QFM[i])\n        Psi_sum += np.outer(norm_state,np.conj(norm_state))\n    #state = np.sum(train_QFM,axis=0)\n    #norm_state = state/np.sum(state)\n    density = Psi_sum/len(train_QFM)\n    return density\n\nprint(np.shape(np.outer(np.zeros(3),np.zeros(3))))\n'

In [ ]:
# Obtaining the density matrix
def mixed_density(train_QFM):
    rho_matrix = np.zeros((np.shape(train_QFM)[1], np.shape(train_QFM)[1]), dtype=np.complex128)
    for state in train_QFM:
        norm_state = state/np.linalg.norm(state)
        rho_matrix += np.outer(norm_state,np.conjugate(norm_state))
    rho_matrix = rho_matrix/np.shape(train_QFM)[0]
    return rho_matrix

# Analysis over matrices

In [ ]:
def eigen(matrix):
    eigenvalues, eigenvectors = np.linalg.eigh(matrix)
    return eigenvalues, eigenvectors
'''''''''
def plot_spectra(matrix, log_xscale = True, log_yscale = False):
    eigenvalues, eigenvectors = np.linalg.eigh(matrix)
    plt.figure(figsize=(6, 4))
    """
    plt.title('Eigenvalues distribution')
    plt.scatter(np.real(eigenvalues), np.ones(len(eigenvalues)), marker='o', color='b')
    plt.show()
    plt.title('Histogram')
    plt.hist(np.real(eigenvalues), bins=30)
    plt.show()
    """
    plt.title('Absolute distribution')
    x_arr = np.logspace(-10, 0, 1000)
    abs_dist = lambda x: [np.sum(eigenvalues <= x_i) for x_i in x]
    #print(np.shape(abs_dist(x_arr)))
    #print(np.shape(x_arr))
    #print(np.shape(np.sum(abs_dist(x_arr), axis=1)))
    abs_distribution = abs_dist(x_arr)
    plt.scatter(x_arr, abs_dist(x_arr))
    threshold = 0.99
    idx = np.argmax( abs_distribution/abs_distribution[-1]>= threshold)
    x_value = x_arr[idx]
    plt.axvline(x=x_value, color='r', linestyle='--', label=f'{threshold*100:.1f}% Threshold at x={x_value:.4f}')
    plt.legend()
    if log_xscale: plt.xscale('log')
    if log_yscale: plt.yscale('log')
    plt.xlabel('Eigenvalue')
    plt.ylabel('Number of Eigenvalues')
    plt.show()
    #return eigenvalues, eigenvectors
'''''''''

'\ndef plot_spectra(matrix, log_xscale = True, log_yscale = False):\n    eigenvalues, eigenvectors = np.linalg.eigh(matrix)\n    plt.figure(figsize=(6, 4))\n    """\n    plt.title(\'Eigenvalues distribution\')\n    plt.scatter(np.real(eigenvalues), np.ones(len(eigenvalues)), marker=\'o\', color=\'b\')\n    plt.show()\n    plt.title(\'Histogram\')\n    plt.hist(np.real(eigenvalues), bins=30)\n    plt.show()\n    """\n    plt.title(\'Absolute distribution\')\n    x_arr = np.logspace(-10, 0, 1000)\n    abs_dist = lambda x: [np.sum(eigenvalues <= x_i) for x_i in x]\n    #print(np.shape(abs_dist(x_arr)))\n    #print(np.shape(x_arr))\n    #print(np.shape(np.sum(abs_dist(x_arr), axis=1)))\n    abs_distribution = abs_dist(x_arr)\n    plt.scatter(x_arr, abs_dist(x_arr))\n    threshold = 0.99\n    idx = np.argmax( abs_distribution/abs_distribution[-1]>= threshold)\n    x_value = x_arr[idx]\n    plt.axvline(x=x_value, color=\'r\', linestyle=\'--\', label=f\'{threshold*100:.1f}% Threshold at x={x_

In [ ]:
np.sum(np.array([1,2,3,4]) <= 2)

2

In [ ]:
def plot_spectra(matrix, log_yscale=False, trunc_at = None):
    '''
    matrix to plot the spectra
    log_yscale = False, True if want to try potential
    trunc_at = None, from zero to one, fraction of eigenvalues to plot
    '''
    # Calculate eigenvalues and eigenvectors
    eigenvalues, _ = np.linalg.eigh(matrix)

    # Set up the plot
    plt.figure(figsize=(6, 4))
    plt.title('Absolute Distribution')

    # Absolute distribution function
    x_arr = np.logspace(-10, 0, 1000)
    abs_dist = lambda x: np.array([np.sum(eigenvalues <= x_i) for x_i in x])
    abs_distribution = abs_dist(x_arr)

    if trunc_at is not None:
        trunc_int = int(trunc_at*len(x_arr))
        x_arr = x_arr[:trunc_int]
        abs_distribution = abs_distribution[:trunc_int]

    # Scatter plot of the distribution
    plt.scatter(x_arr, abs_distribution, label='Data', alpha=0.7)

    if log_yscale:
        # Log-log regression: y = ax^b -> log(y) = log(a) + b*log(x)
        log_x = np.log(x_arr)
        log_y = np.log(abs_distribution)
        popt, pcov = np.polyfit(log_x, log_y, 1, cov=True) # Linear fit in log-log space
        a = np.exp(popt[1])
        b = popt[0]
        regression_label = f'y = {a:.2e} * x^{b:.2f}'
        plt.plot(x_arr, a * x_arr**b, label=f'Fit: {regression_label}', color='r', linestyle='-')

        # Calculate uncertainties
        a_uncertainty = np.sqrt(pcov[1, 1]) * np.exp(popt[1])
        b_uncertainty = np.sqrt(pcov[0, 0])

        print(f"a = {a:.2e} ± {a_uncertainty:.2e}")
        print(f"b = {b:.2f} ± {b_uncertainty:.2f}")

        # Correlation
        correlation_coefficient = np.corrcoef(log_x, log_y)[0, 1]
        print(f"Correlation coefficient (r) = {correlation_coefficient:.5f}")

    if not log_yscale:
        # Log-linear regression: y = ae^(bx) -> log(y) = log(a) + bx
        log_x = np.log(x_arr)
        popt, pcov = np.polyfit(log_x, abs_distribution, 1, cov=True)  # Linear fit in log(y) vs x
        a = popt[1]
        b = popt[0]

        regression_label = f'y = {b:.2e} * log(x) + {a:.2e}'
        plt.plot(x_arr, b * log_x + a, label=f'Fit: {regression_label}', color='r', linestyle='-')


        # Calculate uncertainties
        a_uncertainty = np.sqrt(pcov[1, 1])
        b_uncertainty = np.sqrt(pcov[0, 0])

        print(f"a = {a:.2f} ± {a_uncertainty:.2f}")
        print(f"b = {b:.2f} ± {b_uncertainty:.2f}")

        # Correlation
        correlation_coefficient = np.corrcoef(log_x, abs_distribution)[0, 1]
        print(f"Correlation coefficient (r) = {correlation_coefficient:.5f}")
    """
    # Threshold line
    threshold = 0.99
    idx = np.argmax(abs_distribution / abs_distribution[-1] >= threshold)
    x_value = x_arr[idx]
    plt.axvline(x=x_value, color='g', linestyle='--', label=f'{threshold*100:.1f}% Threshold at x={x_value:.4f}')
    """
    # Set scale, labels, and legend
    plt.xscale('log')
    if log_yscale: plt.yscale('log')
    plt.xlabel('Eigenvalue')
    plt.ylabel('Number of Eigenvalues')
    plt.legend()
    plt.show()

In [ ]:
def trace(matrix):
    return np.trace(matrix)

# Measurement functions

In [ ]:
def make_QFM(quantum_feature_map, N_qfm, param_qfm, train_arr, train_lb, test_arr):
    ## Quantum Feature Map of the train and test sets
    train_QFM_x = np.array([quantum_feature_map(train_point,N_qfm,param_qfm) for train_point in train_arr])
    train_QFM_x0 = train_QFM_x[np.where(train_lb == 0)]
    train_QFM_x1 = train_QFM_x[np.where(train_lb == 1)]

    test_QFM_x = np.array([quantum_feature_map(test_point,N_qfm,param_qfm) for test_point in test_arr])

    return train_QFM_x0, train_QFM_x1, test_QFM_x

In [ ]:
def make_grid(quantum_feature_map, N_qfm, param_qfm,N_grid):
    # Generate grid of x and y values
    xgrid = np.linspace(0, 1, N_grid)
    ygrid = np.linspace(0, 1, N_grid)
    X, Y = np.meshgrid(xgrid, ygrid)  # Create a 2D grid

    # Flatten the meshgrid arrays to pair all (x, y) points
    grid_xy = np.column_stack((X.ravel(), Y.ravel()))
    #print(grid_xy[0])

    # Compute the qfm for each (x, y) pair
    grid_QFM = np.array([quantum_feature_map(point_xy,N_qfm,param_qfm) for point_xy in grid_xy])
    #print('QFM of grid')
    #print(grid_QFM[:10])

    return grid_QFM, grid_xy


In [ ]:
def measurement_pure( train_QFM_x0, train_QFM_x1, test_QFM_x, test_lb, grid_QFM):

    ## Make the pure state
    pstate_x0 = pure_state(train_QFM_x0)
    pstate_x1 = pure_state(train_QFM_x1)
    #print(pstate_x0)

    #P_x0 = np.zeros(len(test_QFM_x))
    #P_x1 = np.zeros(len(test_QFM_x))
    # Compute the dot products in a vectorized manner
    P_x0 = np.abs(np.dot(test_QFM_x.conj(), pstate_x0))**2
    P_x1 = np.abs(np.dot(test_QFM_x.conj(), pstate_x1))**2

    # Normalize the probabilities
    P_norm = P_x0 + P_x1
    P_x0_nrmld = P_x0 / P_norm
    P_x1_nrmld = P_x1 / P_norm

    #Lets check the nan values and find the accuracy without them
    nan_mask = np.isnan(P_x0_nrmld) | np.isnan(P_x1_nrmld) # I think is not neccesary the |
    if np.sum(nan_mask)>0: print(f'There are {np.sum(nan_mask)} nan values')

    P_x0_nrmld = P_x0_nrmld[~nan_mask]
    P_x1_nrmld = P_x1_nrmld[~nan_mask]
    test_lb = test_lb[~nan_mask]

    # Now
    predicted_labels = np.array(P_x1_nrmld>0.5).astype(int)
    #print(predicted_labels)

    # Calculate accuracy
    accuracy = np.mean(predicted_labels == test_lb)
    #print(f"Accuracy: {accuracy}")
    """
    #print(f"Accuracy: {accuracy}")
    print(f'N of predicted = {np.sum(predicted_labels == test_lb)}.')
    print(f'N of test = {len(test_lb)}.')
    print(f'Length of predicted = {len(predicted_labels)}.')
    print(np.array(predicted_labels==test_lb).astype(int))
    print(np.arange(len(test_lb))[predicted_labels!=test_lb])
    """
    ## Plot
    #P_x0_grid = np.zeros(len(grid_QFM))
    #P_x1_grid = np.zeros(len(grid_QFM))
    # Compute the dot products in a vectorized manner for grid
    P_x0_grid = np.abs(np.dot(grid_QFM.conj(), pstate_x0))**2
    P_x1_grid = np.abs(np.dot(grid_QFM.conj(), pstate_x1))**2

    return P_x0, P_x1, P_x0_grid, P_x1_grid, accuracy

In [ ]:
#np.dot(np.array([[1,2,3],[1,2,4]]),np.array([1,2,3]))

In [ ]:
def measurement_mixed( train_QFM_x0, train_QFM_x1, test_QFM_x, test_lb, grid_QFM, density_x0 = None, density_x1 = None):

    if density_x0 is None and density_x1 is None:
        mdensity_x0 = mixed_density(train_QFM_x0)
        mdensity_x1 = mixed_density(train_QFM_x1)
    else:
        mdensity_x0 = density_x0
        mdensity_x1 = density_x1

    #P_x0 = np.zeros(len(test_QFM_x))
    #P_x1 = np.zeros(len(test_QFM_x))
    # Compute Pm_x0 and Pm_x1 in a vectorized manner (be careful)
    P_x0 = np.abs(np.einsum('ij,ij->i', np.dot(test_QFM_x.conj(), mdensity_x0), test_QFM_x))
    P_x1 = np.abs(np.einsum('ij,ij->i', np.dot(test_QFM_x.conj(), mdensity_x1), test_QFM_x))

    """
    # The old version

    # Initialize arrays to store the results
    P_x0_grid = np.zeros(len(grid_QFM))
    P_x1_grid = np.zeros(len(grid_QFM))

    # Compute the double dot product for each vector v in grid_QFM
    for i, v in enumerate(grid_QFM):
        P_x0_grid[i] = np.abs(np.vdot(v, np.dot(mdensity_x0, v)))
        P_x1_grid[i] = np.abs(np.vdot(v, np.dot(mdensity_x1, v)))
    """
    # Normalize probabilities
    normP = (P_x0 + P_x1)
    P_x0_nrmld  = P_x0 / normP
    P_x1_nrmld  = P_x1 / normP

    #Lets check the nan values and find the accuracy without them
    nan_mask = np.isnan(P_x0_nrmld) | np.isnan(P_x1_nrmld) # I think is not neccesary the |
    if np.sum(nan_mask)>0: print(f'There are {np.sum(nan_mask)} nan values')

    P_x0_nrmld = P_x0_nrmld[~nan_mask]
    P_x1_nrmld = P_x1_nrmld[~nan_mask]
    test_lb = test_lb[~nan_mask]

    # Now
    predicted_labels = np.array(P_x1_nrmld>0.5).astype(int)
    #print(predicted_labels)

    # Calculate accuracy
    accuracy = np.mean(predicted_labels == test_lb)
    #print(f"Accuracy: {accuracy}")
    """
    #print(f"Accuracy: {accuracy}")
    print(f'N of predicted = {np.sum(predicted_labels == test_lb)}.')
    print(f'N of test = {len(test_lb)}.')
    print(f'Length of predicted = {len(predicted_labels)}.')
    print(np.array(predicted_labels==test_lb).astype(int))
    print(np.arange(len(test_lb))[predicted_labels!=test_lb])
    """
    ## Plot
    #P_x0_grid = np.zeros(len(grid_QFM))
    #P_x1_grid = np.zeros(len(grid_QFM))
    # Compute the dot products in a vectorized manner for grid
    P_x0_grid = np.abs(np.einsum('ij,ij->i', np.dot(grid_QFM.conj(), mdensity_x0), grid_QFM))
    P_x1_grid = np.abs(np.einsum('ij,ij->i', np.dot(grid_QFM.conj(), mdensity_x1), grid_QFM))

    # Be careful, the conjugate must be in the first grid_QFM, I don't know why


    return P_x0, P_x1, P_x0_grid, P_x1_grid, accuracy, mdensity_x0, mdensity_x1

In [ ]:
def measurement_diagonal( train_QFM_x0, train_QFM_x1, test_QFM_x, test_lb, grid_QFM, density_x0 = None, density_x1 = None):

    if density_x0 is None and density_x1 is None:
        density_x0 = mixed_density(train_QFM_x0)
        density_x1 = pure_density(train_QFM_x1)

    mdiagonal_x0 = density_x0.diagonal()
    mdiagonal_x1 = density_x1.diagonal()

    # Compute the dot products in a vectorized manner
    squared_test_QFM_x = np.square(np.abs(test_QFM_x))
    P_x0 = np.abs(np.dot(squared_test_QFM_x, mdiagonal_x0))
    P_x1 = np.abs(np.dot(squared_test_QFM_x, mdiagonal_x1))

    # Normalize probabilities
    normP = (P_x0 + P_x1)
    P_x0_nrmld  = P_x0 / normP
    P_x1_nrmld  = P_x1 / normP

    #Lets check the nan values and find the accuracy without them
    nan_mask = np.isnan(P_x0_nrmld) | np.isnan(P_x1_nrmld) # I think is not neccesary the |
    if np.sum(nan_mask)>0: print(f'There are {np.sum(nan_mask)} nan values')

    P_x0_nrmld = P_x0_nrmld[~nan_mask]
    P_x1_nrmld = P_x1_nrmld[~nan_mask]
    test_lb = test_lb[~nan_mask]

    # Now
    predicted_labels = np.array(P_x1_nrmld>=0.5).astype(int)
    #print(predicted_labels)
    #accuracy = 0
    # Calculate accuracy
    accuracy = np.mean(predicted_labels == test_lb)
    """
    #print(f"Accuracy: {accuracy}")
    print(f'N of predicted = {np.sum(predicted_labels == test_lb)}.')
    print(f'N of test = {len(test_lb)}.')
    print(f'Length of predicted = {len(predicted_labels)}.')
    print(f'Is predicted? {np.array(predicted_labels==test_lb).astype(int)}')
    print(f'Which bad predicted {np.arange(len(test_lb))[predicted_labels!=test_lb]}')
    print(f'Predicted labels {predicted_labels}')
    #print(f'Test labels {test_lb}')
    #print(f'test_QFM {test_QFM_x}')
    #print(f'squared_test_QFM_x {squared_test_QFM_x}')
    print(f'mdiagonal_x0 {mdiagonal_x0}')
    print(f'mdiagonal_x1 {mdiagonal_x1}')
    """

    ## Plot
    P_x0_grid = np.zeros(len(grid_QFM))
    P_x1_grid = np.zeros(len(grid_QFM))


    # Compute the dot products in a vectorized manner for grid
    squared_grid_QFM_x = np.square(np.abs(grid_QFM))
    P_x0_grid = np.abs(np.dot(squared_grid_QFM_x, mdiagonal_x0))
    P_x1_grid = np.abs(np.dot(squared_grid_QFM_x, mdiagonal_x1))

    return P_x0, P_x1, P_x0_grid, P_x1_grid, accuracy

In [ ]:
def plot_clasification(grid_xy, test_arr, test_lb, P_x0, P_x1, accuracy, dataset, quantum_feature_map, type_est):
    ## Individual
    # Create a red, white, blue colormap
    cmap_blue = LinearSegmentedColormap.from_list('white_blue', ['white', 'blue'])
    colors_blue = P_x0
    cmap_red = LinearSegmentedColormap.from_list('white_red', ['white', 'red'])
    colors_red = P_x1

    # Scatter plot with the proper grid
    plt.figure(figsize=(9, 4))
    plt.subplot(121)
    sc1= plt.scatter(grid_xy[:, 0],grid_xy[:, 1], c=colors_blue,
                cmap=cmap_blue, marker='.')
    plt.colorbar(sc1)
    plt.scatter(test_arr[test_lb ==0, 0], test_arr[test_lb ==0, 1],
                c = 'b', edgecolor='k', label=dataset)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.subplot(122)
    sc2 = plt.scatter(grid_xy[:, 0], grid_xy[:, 1], c=colors_red,
                cmap=cmap_red, marker='.')
    plt.colorbar(sc2)
    plt.scatter(test_arr[test_lb ==1, 0], test_arr[test_lb ==1, 1],
                c = 'r', edgecolor='k', label=dataset)
    plt.xlabel('x')
    plt.suptitle('Individual '+ (quantum_feature_map.__name__) + ' ' + type_est)
    plt.show()

    Pnorm_x0 = P_x0 / (P_x0 + P_x1)
    ## Joint
    # Create a red, white, blue colormap
    cmap = LinearSegmentedColormap.from_list('red_white_blue', ['red', 'white', 'blue'])
    colors = Pnorm_x0

    # Scatter plot with the proper grid
    plt.figure(figsize=(6, 6))
    sc = plt.scatter(grid_xy[:, 0], grid_xy[:, 1], c=colors, cmap=cmap)

    plt.scatter(test_arr[:, 0], test_arr[:, 1], c=test_lb, cmap='bwr', edgecolor='k', label=dataset)

    # Annotate all test points with indices
    #for i, (x, y) in enumerate(test_arr):
    #    plt.text(x, y, str(i), fontsize=8, color='black')

    # Labels and title
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Joint '+ (quantum_feature_map.__name__) + ' ' + type_est+ ' Accuracy: '+str(accuracy))

    # Show the plot
    plt.show()

# Reduce Matrix

In [ ]:
'''''''''

def reduce_matrix(matrix):
    # Step 1: Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eigh(matrix)

    # Step 2: Truncate eigenvalues below a threshold
    threshold = 2.0
    reduced_eigenvalues = np.where(eigenvalues > threshold, eigenvalues, 0)

    # Step 3: Reconstruct the matrix
    # D is the diagonal matrix of reduced eigenvalues
    D_reduced = np.diag(reduced_eigenvalues)
    matrix_reduced = eigenvectors @ D_reduced @ eigenvectors.T
    return matrix_reduced

'''''''''

'\n\ndef reduce_matrix(matrix):\n    # Step 1: Compute eigenvalues and eigenvectors\n    eigenvalues, eigenvectors = np.linalg.eigh(matrix)\n\n    # Step 2: Truncate eigenvalues below a threshold\n    threshold = 2.0\n    reduced_eigenvalues = np.where(eigenvalues > threshold, eigenvalues, 0)\n\n    # Step 3: Reconstruct the matrix\n    # D is the diagonal matrix of reduced eigenvalues\n    D_reduced = np.diag(reduced_eigenvalues)\n    matrix_reduced = eigenvectors @ D_reduced @ eigenvectors.T\n    return matrix_reduced\n\n'

In [ ]:
def reduce_matrix(matrix, threshold = 0):
    '''
    threshold = 0,  between 0 and 1
    '''
    # Step 1: Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eigh(matrix)

    # Step 2: Truncate eigenvalues below a threshold
    max_eigenvalue = np.max(eigenvalues)
    threshold2 = max_eigenvalue * threshold
    reduced_eigenvalues = np.where(eigenvalues > threshold2, eigenvalues, 0)
    ### The commented code below were just to try an hypotesis
    #min = 0.1
    #max = 1
    #mask = (eigenvalues >= min*max_eigenvalue) & (eigenvalues <= max*max_eigenvalue)
    #reduced_eigenvalues = np.where(mask, eigenvalues, 0)
    nonzero_indices = np.nonzero(reduced_eigenvalues)
    print(f'Matrix reduced from dimension {len(eigenvalues)} to {len(nonzero_indices[0])}')
    #print(f'Eigenvalues {eigenvalues}')
    # Step 3: Reconstruct the matrix
    # D is the diagonal matrix of reduced eigenvalues
    #reduced_eigenvalues = eigenvalues
    D_reduced = np.diag(reduced_eigenvalues)
    matrix_reduced = eigenvectors @ D_reduced @ eigenvectors.T.conj()
    return matrix_reduced